In [1]:
import sys, os
import argparse

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../../")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../")))

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "")))

In [2]:
from fedml_api.data_preprocessing.cifar10.data_loader import load_partition_data_cifar10
from fedml_api.standalone.fedavg.my_model_trainer_classification import MyModelTrainer as MyModelTrainerCLS
from fedml_api.model.contrastive_cv.resnet_with_embedding import Resnet56
from CovaMNet2 import CovaMResnet56

import torch
from torch import nn
from collections import OrderedDict
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import random
import pickle
import wandb

from pcgrad import PCGrad

# from triplet_loss import TripletLoss
from hard_triplet_loss import TripletLoss

<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject


In [3]:
dataset = 'cifar10'
data_dir = "./../../../data/cifar10"
partition_method = 'hetero'
# partition_method = 'homo'
partition_alpha = 0.5
client_num_in_total = 10
batch_size = 100
total_epochs = 500

save_model_path = 'model/cs_{0}_{1}_client_{2}_variable_covaM_epochs_{3}.pt'

device = 'cuda:1'
with_cova = True

wandb.init(
            # project="federated_nas",
            project="cova",
            name="Cova_" + str(partition_method) + "_clients" + str(client_num_in_total) + "_pcgrad_wn_2:1_adam-leaky_relu_without_mean_with_cova" + str(
                with_cova)
        )
# config = wandb.config
# config.learning_rate = 0.01

with open(f'dataset_{partition_method}_{client_num_in_total}.pickle', 'rb') as f:
    dataset = pickle.load(f)
# [train_data_num, test_data_num, train_data_global, test_data_global, \
# #             train_data_local_num_dict, train_data_local_dict, test_data_local_dict, \
# #             class_num]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: slimfun (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
# from collections import Counter
# ls = np.array([int(l) for l in labels])
# counter_result = Counter(ls)
# print(Counter(ls))
# print(dataset[5])

In [5]:
# model = Resnet56(class_num=dataset[-1], neck='bnneck')
# model.load_state_dict(torch.load('model/cs_3_homo_client_0_oral_epochs_200.pt'))

In [6]:
with open(f'data_dict.pickle', 'rb') as f:
    data_dict = pickle.load(f)

In [7]:
cl = []
for k, v in data_dict.items():
#     print(len(v))
    cl.append(torch.stack(v).to(device))
# print(cl[0].shape)
print(len(cl))

10


In [8]:
# def cal_covariance(input):

# #     CovaMatrix_list = []
# #     mean_list = []
#     B, C, h, w = input.size()
# #     print(B)
#     support_set_sam = input.permute(1, 0, 2, 3)
#     support_set_sam = support_set_sam.contiguous().view(C, -1)
    
#     mean_support = torch.mean(support_set_sam, 1, True)
# #     mean_list.append(mean_support)
    
#     support_set_sam = support_set_sam-mean_support
    
#     covariance_matrix = support_set_sam@torch.transpose(support_set_sam, 0, 1)
#     covariance_matrix = torch.div(covariance_matrix, h*w*B-1)
    
#     return covariance_matrix.clone().detach(), mean_support.clone().detach()


# def get_cova(model, device):
#     cova_list = [0 for i in range(10)]
#     mean_list = [0 for i in range(10)]
    
#     model.to(device)
#     model.eval()
    
#     features = []
    
#     with torch.no_grad():
#         for k, v in data_dict.items():
#             x = torch.stack(data_dict[k]).to(device)
# #             print(x.shape)
#             score, feats = model(x)
#             covariance_matrix, mean_support = cal_covariance(feats.to(device))
#             cova_list[k] = covariance_matrix.to(device)
#             mean_list[k] = mean_support.to(device)
            
#     return cova_list, mean_list


In [9]:
class Client(object):
    def __init__(self, client_index, train_data_local_dict, train_data_local_num_dict, test_data_local_dict, device, model):
        self.id = client_index
        self.train_data = train_data_local_dict[self.id]
        self.local_sample_number = train_data_local_num_dict[self.id]
        self.test_local = test_data_local_dict[self.id]
        
        self.device = device
        self.model = model
model1 = CovaMResnet56(class_num=dataset[-1], neck='bnneck', with_cova=with_cova)
client_1 = Client(0, dataset[5], dataset[4], dataset[6], device, model1)

In [10]:
def test(model, test_data, device, cl, ml):
#     cl = cms[0]
#     ml = cms[1]
    
    model.to(device)
    model.eval()

    metrics = {
        'test_correct': 0,
        'test_loss': 0,
        'test_total': 0,
        'correct_cova': 0
    }

    criterion = nn.CrossEntropyLoss().to(device)

    with torch.no_grad():
        for batch_idx, (x, target) in enumerate(test_data):
            x = x.to(device)
            target = target.to(device)
#             support_ml = []
#             for l in target:
#                 support_ml.append(ml[l.data].to(device))
            pred, pred_ce_cova, feat = model(x, cl, ml=None)
            loss = criterion(pred, target)
            
#             loss_ce_cova = criterion(pred_ce_cova, target)

            _, predicted = torch.max(pred, -1)
            _, predicted_ce_cova = torch.max(pred_ce_cova, -1)
            correct = predicted.eq(target).sum()
            correct_ce_cova = predicted_ce_cova.eq(target).sum()

            metrics['test_correct'] += correct.item()
#             metrics['test_correct'] += correct_ce_cova.item()
            metrics['test_loss'] += loss.item() * target.size(0)
            metrics['test_total'] += target.size(0)
            metrics['correct_cova'] += correct_ce_cova.item()
            
    return metrics

def train_model(client, epochs):
#     learning_rate = 0.001
#     wd = 0.0001
#     learning_rate = 0.00035
#     wd = 0.0005

#     cl = cms[0]
#     ml = cms[1]
    
    
    margin = 0.3
    
    client.model.to(client.device)
    client.model.train()
    
    criterion = nn.CrossEntropyLoss().to(device)
    criterion2 = nn.CrossEntropyLoss().to(device)
    mse_loss = nn.MSELoss().to(device)
    
#     curr_lr = learning_rate
#     optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, client.model.parameters()), lr=curr_lr,
#                                          weight_decay=wd, amsgrad=True)
#     optimizer = torch.optim.SGD(client.model.parameters(), lr=0.001,
#                       momentum=0.9, weight_decay=5e-4)
    optimizer = PCGrad(torch.optim.Adam(client.model.parameters(), lr=0.001, betas=(0.5, 0.9)))
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

    epoch_loss = []
    total_step = len(client.train_data)
#     wandb.watch(client.model, log='all')
    lbd = 0.1
    for epoch in range(epochs):
        
#         if epoch > 0:
        
#         print(ml[0])
        
        client.model.train()
        batch_loss = []
        for batch_idx, (x, labels) in enumerate(client.train_data):
            
#             cl, ml = get_cova(client.model, client.device)
#             if epoch > 0:
#                 print(cl[0])
            x, labels = x.to(device), labels.to(device)
#             print(labels.shape)
            client.model.zero_grad()
#             support_ml = []
#             for l in labels:
#                 # B, 256, 64
#                 support_ml.append(ml[l.data].to(device))
#             print(cl[0].shape)
            cls_score, cova_score, feat = client.model(x, support_imgs=cl, ml=None)
#             print(score.shape)
#             print(cova_score)
#             print(cova_score)
#             loss = criterion(cova_score, labels).to(device)
#             if epoch < 100:
#             loss = criterion(cls_score, labels)
            loss1 = 2*criterion(cls_score, labels)
            loss2 = criterion2(cova_score, labels)
            losses = [loss2, loss1]
#             if epoch > 100 :
#             loss = criterion(cova_score, labels).to(device)
#             loss = loss.to(device) + criterion(cova_score, labels).to(device)
            lbd += (1-0.1) / (epochs-100)
#             loss.backward()
            optimizer.pc_backward(losses)
        
#             torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            batch_loss.append((loss1+loss2).item())
        
#         print(cl[0])
            
#         if epoch % 50 == 0:
#             torch.save(client.model.state_dict(), str.format(save_model_path, client_num_in_total, partition_method, client.id, epoch))
        epoch_loss.append(sum(batch_loss) / len(batch_loss))
#         scheduler.step()
        print('Client Index = {}\tEpoch: {}\tLoss: {:.6f}'.format(
            client.id, epoch, sum(batch_loss) / len(batch_loss)))
        
        metrics = test(client.model, client.test_local, client.device, cl, None)
        test_correct = metrics['test_correct']/metrics['test_total']
        test_loss = metrics['test_loss']/metrics['test_total']
        correct_ce_cova = metrics['correct_cova']/metrics['test_total']
#         print(metrics['test_total'])
        print(f'test_correct: {test_correct}; test_loss: {test_loss}')
        wandb.log({
#             'epoch' : epoch,
            'train_loss': sum(batch_loss) / len(batch_loss),
            'test_correct' : test_correct,
            'test_loss' : test_loss,
            'correct_cova' : correct_ce_cova
        }, step=epoch)
        
    torch.save(client.model.state_dict(), str.format(save_model_path, client_num_in_total, partition_method, client.id, epochs))

In [11]:
train_model(client_1, 200)

Client Index = 0	Epoch: 0	Loss: 13.200077
test_correct: 0.2447; test_loss: 2.2710285878181455
Client Index = 0	Epoch: 1	Loss: 6.562004
test_correct: 0.2556; test_loss: 2.400473428964615
Client Index = 0	Epoch: 2	Loss: 5.195245
test_correct: 0.2495; test_loss: 2.3365267586708067
Client Index = 0	Epoch: 3	Loss: 4.661460
test_correct: 0.2811; test_loss: 2.106211576461792
Client Index = 0	Epoch: 4	Loss: 4.444129
test_correct: 0.2816; test_loss: 2.3471780371665956
Client Index = 0	Epoch: 5	Loss: 4.312845
test_correct: 0.3074; test_loss: 2.2855571687221525
Client Index = 0	Epoch: 6	Loss: 4.070860
test_correct: 0.3327; test_loss: 2.0321888267993926
Client Index = 0	Epoch: 7	Loss: 4.020527
test_correct: 0.3173; test_loss: 2.379863735437393
Client Index = 0	Epoch: 8	Loss: 3.757673
test_correct: 0.3131; test_loss: 2.374501119852066
Client Index = 0	Epoch: 9	Loss: 3.684219
test_correct: 0.3503; test_loss: 2.352310218811035
Client Index = 0	Epoch: 10	Loss: 3.578574
test_correct: 0.3276; test_loss:

Client Index = 0	Epoch: 88	Loss: 1.201114
test_correct: 0.5139; test_loss: 2.0743940830230714
Client Index = 0	Epoch: 89	Loss: 1.230933
test_correct: 0.4517; test_loss: 2.6299739515781404
Client Index = 0	Epoch: 90	Loss: 1.122487
test_correct: 0.5316; test_loss: 2.037427456378937
Client Index = 0	Epoch: 91	Loss: 1.142989
test_correct: 0.4847; test_loss: 2.260994894504547
Client Index = 0	Epoch: 92	Loss: 1.148837
test_correct: 0.5059; test_loss: 2.148344153165817
Client Index = 0	Epoch: 93	Loss: 1.097837
test_correct: 0.5512; test_loss: 2.053805807828903
Client Index = 0	Epoch: 94	Loss: 1.084028
test_correct: 0.5242; test_loss: 2.039959878921509
Client Index = 0	Epoch: 95	Loss: 1.078428
test_correct: 0.5139; test_loss: 2.180318487882614
Client Index = 0	Epoch: 96	Loss: 1.156023
test_correct: 0.5319; test_loss: 2.120999276638031
Client Index = 0	Epoch: 97	Loss: 1.052824
test_correct: 0.5097; test_loss: 2.2752139055728913
Client Index = 0	Epoch: 98	Loss: 1.111297
test_correct: 0.5279; tes

Client Index = 0	Epoch: 175	Loss: 0.614789
test_correct: 0.5605; test_loss: 2.5187373518943788
Client Index = 0	Epoch: 176	Loss: 0.645373
test_correct: 0.5488; test_loss: 2.7943439185619354
Client Index = 0	Epoch: 177	Loss: 0.601955
test_correct: 0.5524; test_loss: 2.5406037330627442
Client Index = 0	Epoch: 178	Loss: 0.613792
test_correct: 0.5299; test_loss: 2.614033601284027
Client Index = 0	Epoch: 179	Loss: 0.590165
test_correct: 0.5611; test_loss: 2.4526961207389832
Client Index = 0	Epoch: 180	Loss: 0.522619
test_correct: 0.5715; test_loss: 2.455567146539688
Client Index = 0	Epoch: 181	Loss: 0.588634
test_correct: 0.5527; test_loss: 2.5114580714702606
Client Index = 0	Epoch: 182	Loss: 0.565750
test_correct: 0.5312; test_loss: 2.5880572414398193
Client Index = 0	Epoch: 183	Loss: 0.588659
test_correct: 0.5471; test_loss: 2.436151785850525
Client Index = 0	Epoch: 184	Loss: 0.608814
test_correct: 0.546; test_loss: 2.5394062530994415
Client Index = 0	Epoch: 185	Loss: 0.584867
test_correc